In [23]:
!apt-get purge --auto-remove -y cuda
!rm -rf /usr/local/cuda*

!wget https://developer.download.nvidia.com/compute/cuda/11.8.0/local_installers/cuda_11.8.0_520.61.05_linux.run
!chmod +x cuda_11.8.0_520.61.05_linux.run
!./cuda_11.8.0_520.61.05_linux.run --silent --toolkit

import os
os.environ['PATH'] = '/usr/local/cuda-11.8/bin:' + os.environ['PATH']
os.environ['LD_LIBRARY_PATH'] = '/usr/local/cuda-11.8/lib64:' + os.environ.get('LD_LIBRARY_PATH', '')



Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Package 'cuda' is not installed, so not removed
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.
--2025-06-12 20:34:23--  https://developer.download.nvidia.com/compute/cuda/11.8.0/local_installers/cuda_11.8.0_520.61.05_linux.run
Resolving developer.download.nvidia.com (developer.download.nvidia.com)... 23.43.51.10, 23.43.51.15
Connecting to developer.download.nvidia.com (developer.download.nvidia.com)|23.43.51.10|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4336730777 (4.0G) [application/octet-stream]
Saving to: ‘cuda_11.8.0_520.61.05_linux.run’

cuda_11.8.0_520.61. 100%[===================>]   4.04G   104MB/s    in 57s     

2025-06-12 20:35:20 (72.0 MB/s) - ‘cuda_11.8.0_520.61.05_linux.run’ saved [4336730777/4336730777]

Signal caught, cleaning up
Signal caught, cleaning up
/bin/bash: line 1: nvcc: command not found
Thu Jun 12 20:38:38 2025     

In [24]:
!./cuda_11.8.0_520.61.05_linux.run --silent --toolkit

In [25]:
!nvcc --version
!nvidia-smi

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0
Thu Jun 12 20:43:52 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   43C    P8       

In [26]:
!pip install nvcc4jupyter
%load_ext nvcc4jupyter

The nvcc4jupyter extension is already loaded. To reload it, use:
  %reload_ext nvcc4jupyter


In [22]:

!pip install pycuda
!nvcc --version

  Using cached pycuda-2025.1.1.tar.gz (1.7 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached pytools-2025.1.6-py3-none-any.whl.metadata (2.9 kB)
  Using cached siphash24-1.7-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (3.3 kB)
Using cached pytools-2025.1.6-py3-none-any.whl (95 kB)
Using cached siphash24-1.7-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (105 kB)
  error: subprocess-exited-with-error
  
  × Building wheel for pycuda (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for pycuda
Failed to build pycuda
ERROR: ERROR: Failed to build installable wheels for some pyproject.toml based projects (pycuda)
/bin/bash: line 1: nvcc: command not found


# **Scalar Calculation**

In [27]:
first_name = "Muhammad Taha"
roll_number = "FA22-BCS-119"

name_sum = sum(ord(char) for char in first_name.upper())
roll_number = str(roll_number)
roll_number_sum = sum(ord(char) for char in roll_number)
print(name_sum)
print(roll_number_sum)
S = name_sum ^ roll_number_sum
print(f"Name: {first_name}")
print(f"Name ASCII Sum: {name_sum}")
print(f"Roll Number: {roll_number}")
print(f"Roll number ASCII Sum: {roll_number_sum}")
print(f"{name_sum} XOR {roll_number_sum} = {S}")


904
696
Name: Muhammad Taha
Name ASCII Sum: 904
Roll Number: FA22-BCS-119
Roll number ASCII Sum: 696
904 XOR 696 = 304


# **Cuda Implementation**


In [31]:
%%writefile matrix_reduction_simple.cu

#include <cuda_runtime.h>
#include <device_launch_parameters.h>
#include <stdio.h>
#include <stdlib.h>
#include <math.h>

#define MATRIX_SIZE 512
#define BLOCK_SIZE 256

#define PERSONALIZED_S 304

// Macro to check CUDA errors
#define CUDA_CHECK(ans) { gpuAssert((ans), __FILE__, __LINE__); }
inline void gpuAssert(cudaError_t code, const char *file, int line, bool abort=true)
{
   if (code != cudaSuccess)
   {
      fprintf(stderr,"GPUassert: %s %s %d\n", cudaGetErrorString(code), file, line);
      if (abort) exit(code);
   }
}

__global__ void conditionalRowReduction(int* matrix, int* result, int S, int matrixSize){
  extern __shared__ int sdata[];

  int tid = threadIdx.x;
  int row = blockIdx.x;

  sdata[tid] = 0;

  int localSum = 0;
  for(int col=tid; col < matrixSize; col+=blockDim.x){
    if((row * col)%S<10){
      localSum += matrix[row * matrixSize + col];
      }
    }
    sdata[tid] = localSum;
    // Perform standard shared memory reduction
    // This avoids bank conflicts by using power-of-2 strides
    for (int stride = blockDim.x / 2; stride > 0; stride >>= 1) {
        if (tid < stride) {
            sdata[tid] += sdata[tid + stride];
        }
        __syncthreads();
    }

    // Thread 0 writes the final result
    if (tid == 0) {
        result[row] = sdata[0];
    }
  }
  void initializeMatrix(int* matrix, int S, int size) {
    for (int i = 0; i < size; i++) {
        for (int j = 0; j < size; j++) {
            matrix[i * size + j] = ((i + j) * S) % 100;
        }
    }
}
long long calculateTotalSum(int* result, int size) {
    long long total = 0;
    for (int i = 0; i < size; i++) {
        total += result[i];
    }
    return total;
}
int main() {
    // Use hardcoded personalized scalar
    int S = PERSONALIZED_S;

    printf("=== CUDA Conditional Row-wise Matrix Reduction ===\\n");
    printf("Matrix Size: %dx%d\\n", MATRIX_SIZE, MATRIX_SIZE);
    printf("Block Size: %d threads\\n", BLOCK_SIZE);
    printf("Using personalized scalar S\n\n");

    // Host memory allocation
    size_t matrixBytes = MATRIX_SIZE * MATRIX_SIZE * sizeof(int);
    size_t resultBytes = MATRIX_SIZE * sizeof(int);

    int* h_matrix = (int*)malloc(matrixBytes);
    int* h_result = (int*)malloc(resultBytes);

    if (!h_matrix || !h_result) {
        printf("Host memory allocation failed!\n");
        return -1;
    }

    // Initialize matrix on host
    printf("Initializing matrix...\n");
    initializeMatrix(h_matrix, S, MATRIX_SIZE);

    // Device memory allocation
    int* d_matrix;
    int* d_result;

    CUDA_CHECK(cudaMalloc(&d_matrix, matrixBytes));
    CUDA_CHECK(cudaMalloc(&d_result, resultBytes));

    // Transfer matrix to GPU
    printf("Transferring data to GPU...\n");
    CUDA_CHECK(cudaMemcpy(d_matrix, h_matrix, matrixBytes, cudaMemcpyHostToDevice));

    // Configure kernel launch parameters
    dim3 gridSize(MATRIX_SIZE, 1, 1);    // 512 blocks (one per row)
    dim3 blockSize(BLOCK_SIZE, 1, 1);    // 256 threads per block
    size_t sharedMemSize = BLOCK_SIZE * sizeof(int);

    // Create CUDA events for precise timing
    cudaEvent_t start, stop;
    CUDA_CHECK(cudaEventCreate(&start));
    CUDA_CHECK(cudaEventCreate(&stop));

    // Warm up the GPU to get accurate timing
    conditionalRowReduction<<<gridSize, blockSize, sharedMemSize>>>(d_matrix, d_result, S, MATRIX_SIZE);
    CUDA_CHECK(cudaDeviceSynchronize());

    // Measure kernel execution time
    printf("Executing CUDA kernel...\\n");
    CUDA_CHECK(cudaEventRecord(start));

    conditionalRowReduction<<<gridSize, blockSize, sharedMemSize>>>(d_matrix, d_result, S, MATRIX_SIZE);

    CUDA_CHECK(cudaEventRecord(stop));
    CUDA_CHECK(cudaEventSynchronize(stop));

    // Calculate execution time
    float kernelTime;
    CUDA_CHECK(cudaEventElapsedTime(&kernelTime, start, stop));

    // Check for kernel execution errors
    CUDA_CHECK(cudaGetLastError());
    CUDA_CHECK(cudaDeviceSynchronize());

    // Transfer results back to host
    printf("Transferring results back to host...\n");
    CUDA_CHECK(cudaMemcpy(h_result, d_result, resultBytes, cudaMemcpyDeviceToHost));

    // Calculate total sum of result array
    long long totalSum = calculateTotalSum(h_result, MATRIX_SIZE);

    // Print required results
    printf("\\n=== RESULTS ===\\n");
    printf("R[10]  = %d\\n", h_result[10]);
    printf("R[100] = %d\\n", h_result[100]);
    printf("R[200] = %d\\n", h_result[200]);
    printf("Total sum of R array = %lld\n", totalSum);
    printf("\n=== PERFORMANCE METRICS ===\n");
    printf("Kernel execution time = %.4f ms\n", kernelTime);

    // Calculate effective bandwidth
    double totalBytes = matrixBytes + resultBytes;
    double bandwidth = totalBytes / (kernelTime * 1e-3) / 1e9;
    printf("Effective bandwidth = %.2f GB/s\n", bandwidth);

    // Calculate throughput
    long long totalOps = (long long)MATRIX_SIZE * MATRIX_SIZE;
    double throughput = totalOps / (kernelTime * 1e-3) / 1e9;
    printf("Throughput = %.2f GOPS\n", throughput);

    // Optional: Show sample of results for verification
    printf("\n=== VERIFICATION SAMPLE ===\n");
    printf("First 5 row sums: ");
    for (int i = 0; i < 5; i++) {
        printf("%d ", h_result[i]);
    }
    printf("\n");

    // Cleanup memory
    free(h_matrix);
    free(h_result);
    CUDA_CHECK(cudaFree(d_matrix));
    CUDA_CHECK(cudaFree(d_result));
    CUDA_CHECK(cudaEventDestroy(start));
    CUDA_CHECK(cudaEventDestroy(stop));

    printf("\nProgram completed successfully!\n");
    return 0;
}

Overwriting matrix_reduction_simple.cu


In [32]:
!nvcc -o matrix_reduction matrix_reduction_simple.cu -arch=sm_60 -O3

In [33]:
!./matrix_reduction

=== CUDA Conditional Row-wise Matrix Reduction ===\nMatrix Size: 512x512\nBlock Size: 256 threads\nUsing personalized scalar S

Initializing matrix...
Transferring data to GPU...
Executing CUDA kernel...\nTransferring results back to host...
\n=== RESULTS ===\nR[10]  = 844\nR[100] = 612\nR[200] = 1108\nTotal sum of R array = 524552

=== PERFORMANCE METRICS ===
Kernel execution time = 0.0245 ms
Effective bandwidth = 42.81 GB/s
Throughput = 10.68 GOPS

=== VERIFICATION SAMPLE ===
First 5 row sums: 24264 600 360 532 672 

Program completed successfully!
